In [2]:
import pyomo.environ as pyo
from itertools import product

commands = ['c0', 'c1', 'c2', 'c3']
frequencies = {
    'c0': 10,
    'c1': 5,
    'c2': 3,
    'c3': 2
}

slots = [0, 1, 2, 3]

cs = list(product(commands, slots))

# Cost is higher if a more frequently used command is placed in a lower slot
costs = {(command, slot): frequencies[command] * slot for command, slot in cs}

In [3]:
model = pyo.ConcreteModel()
model.x = pyo.Var(commands, slots, bounds=(0, 1))

# Each slot must be filled by exactly one command
def one_slot_rule(mdl, s):
    return sum(mdl.x[c, s] for c in commands) == 1
model.one_slot = pyo.Constraint(slots, rule=one_slot_rule)

# Each command must be assigned to exactly one slot
def one_command_rule(mdl, c):
    return sum(mdl.x[c, s] for s in slots) == 1
model.one_command = pyo.Constraint(commands, rule=one_command_rule)

# Objective function
def objective_func(mdl):

    return sum(costs[c, s] * mdl.x[c, s] for c, s in cs)
model.objective = pyo.Objective(rule=objective_func, sense=pyo.minimize)

In [4]:
solver = pyo.SolverFactory("cbc")  # "cbc", "glpk"
res = solver.solve(model)

pyo.assert_optimal_termination(res)

model.x.display()

x : Size=16, Index=x_index
    Key       : Lower : Value : Upper : Fixed : Stale : Domain
    ('c0', 0) :     0 :   1.0 :     1 : False : False :  Reals
    ('c0', 1) :     0 :   0.0 :     1 : False : False :  Reals
    ('c0', 2) :     0 :   0.0 :     1 : False : False :  Reals
    ('c0', 3) :     0 :   0.0 :     1 : False : False :  Reals
    ('c1', 0) :     0 :   0.0 :     1 : False : False :  Reals
    ('c1', 1) :     0 :   1.0 :     1 : False : False :  Reals
    ('c1', 2) :     0 :   0.0 :     1 : False : False :  Reals
    ('c1', 3) :     0 :   0.0 :     1 : False : False :  Reals
    ('c2', 0) :     0 :   0.0 :     1 : False : False :  Reals
    ('c2', 1) :     0 :   0.0 :     1 : False : False :  Reals
    ('c2', 2) :     0 :   1.0 :     1 : False : False :  Reals
    ('c2', 3) :     0 :   0.0 :     1 : False : False :  Reals
    ('c3', 0) :     0 :   0.0 :     1 : False : False :  Reals
    ('c3', 1) :     0 :   0.0 :     1 : False : False :  Reals
    ('c3', 2) :     0 :   0.